<center><h1>4 DOF Time Domain Model Notebook</h1></center>
<br>
<center><h3>Notebook written by <em>Sreenath S</em></h3></center>

This notebook is an attempt at proper documentation of the development of the 4 DOF Time domain model for **Ship Manoeuvring in Waves**

<p><strong>Jupyter Notebook Shortcuts</strong></p>
<ul>
<li>Toggle between edit and command mode with <code>Esc</code> and <code>Enter</code>, respectively.</li>
<li>Once in command mode:
<ul>
<li>Scroll up and down your cells with your <code>Up</code> and <code>Down</code> keys.</li>
<li>Press <code>A</code> or <code>B</code> to insert a new cell above or below the active cell.</li>
<li><code>M</code> will transform the active cell to a Markdown cell.</li>
<li><code>Y</code> will set the active cell to a code cell.</li>
<li><code>D + D</code> (<code>D</code> twice) will delete the active cell.</li>
<li><code>Z</code> will undo cell deletion.</li>
<li>Hold <code>Shift</code> and press <code>Up</code> or <code>Down</code> to select multiple cells at once.
<ul>
<li>With multiple cells selected, <code>Shift + M</code> will merge your selection.</li>
</ul>
</li>
</ul>
</li>
<li><code>Ctrl + Shift + -</code>, in edit mode, will split the active cell at the cursor.</li>
<li>You can also click and <code>Shift + Click</code> in the margin to the left of your cells to select them.</li>
</ul>

**STEP 1) Import the necessary modules**

In [1]:
# Standard Library Imports

import pandas as pd
import numpy  as np
from math import *
from scipy import constants
import matplotlib.pyplot as plt

# Custom Library Imports

from modules import math_op                           # This custom modules contain several custom math operations

**STEP 2) Read the necessary Input Data**

In [2]:
############################# Intialization ###############################

# 1. Universal Constant

g = constants.g                                       # Acceleration due to gravity ( m/s^2 )
ρ = 1000                                              # Density of fresh water ( kg / m^3 )
π = constants.pi                                      # π

# 2. Hull Properties

df = pd.read_csv('data/vessel_parameters.csv', index_col='parameter', comment='#')

m = df['value']['m']                                  # Ship Mass ( kg )
                         
I_B = np.array([ [df['value']['I_X'],0,0],            # Moment and Product of inertia Matrix ( kg m^2)
                 [0,df['value']['I_Y'],0],
                 [2,0,df['value']['I_Z']] ])

V_disp = df['value']['V_disp']                        # Volume Displacement ( m^3 ). Sea water volume is used.

L   = df['value']['L']                                # Length between perpendiculars ( m )

LCB = df['value']['LCB']                              # Location of LCB from body-fixed CS  ( m )

D = df['value']['D']                                  # Depth of ship ( m )

B = df['value']['B']                                  # Breadth of ship ( m )

d = df['value']['d']                                  # Draft of ship ( m )

GM_T = df['value']['GM_T']                            # Transverse metacentric height ( m )

GM_L = df['value']['GM_L']                            # Longitudinal metacentric height ( m )

R_G = np.array([ [df['value']['X_G']],                # Vector of C.O.G ( m )
                 [df['value']['Y_G']], 
                 [df['value']['Z_G']] ])

Yv_dash = df['value']["Yv'"]                          # First order hydrodynamic derivatives
Yr_dash = df['value']["Yr'"]                          # 
Nv_dash = df['value']["Nv'"]                          #
Nr_dash = df['value']["Nr'"]                          #

Xvr_dash = df['value']["Xvr'"]                        # Second order hydrodynamic derivatives
Xvv_dash = df['value']["Xvv'"]                        # 
Xrr_dash = df['value']["Xrr'"]                        #

Yvvr_dash = df['value']["Yvvr'"]                      # Third order hydrodynamic derivatives 
Yvrr_dash = df['value']["Yvrr'"]                      #
Nvvr_dash = df['value']["Nvvr'"]                      #
Nvrr_dash = df['value']["Nvrr'"]                      # 
Yvvv_dash = df['value']["Yvvv'"]                      #
Yrrr_dash = df['value']["Yrrr'"]                      #
Nvvv_dash = df['value']["Nvvv'"]                      #
Nrrr_dash = df['value']["Nrrr'"]                      #

Xvvvv_dash = df['value']["Xvvvv'"]                    # Fourth order hydrodynamic derivative

# 3. Ship Motion Properties

Tφ = df['value']['Tφ']                                # Ship Natural Roll Period (s)

# 4. Propeller Properties

N_prop = df['value']["N_prop"]                        # No. of Propellers
Dp     = df['value']["Dp"]                            # Propeller diameter ( m )
wp     = df['value']["wp"]                            # Wake Fraction 
tp     = df['value']["tp"]                            # Thrust Deduction Fraction

# 5. Experimental Conditions

Fn = df['value']["Fn"]                                # Froude Number

# 6. R vs Vs Data ( Resistance vs Ship Velocity )

df2 = pd.read_csv('data/R__vs__Vs.csv',  comment='#')
Vs_pts  = df2['Vs'].to_numpy()                        # Vs -> ( m/s ) & Rt -> ( N )
R_pts   = df2['R'].to_numpy()                         # pts stands for points

# 7. Read KT vs J Data ( Thrust Co-eff vs Advance Co-eff. )

df3 = pd.read_csv('data/KT__vs__J.csv',  comment='#')
J_pts  = df3['J'].to_numpy()
KT_pts = df3['KT'].to_numpy()

# 8. Read the calm water GZ data

df4 = pd.read_csv('data/GZ_Data/GZ_calmwater.csv', comment='#')
φ_pts   = df4['φ'].to_numpy() 
GZ_pts  = df4['GZ'].to_numpy()                        # φ -> ( deg ) & GZ -> ( m )

# 9. Test Variables

wind_heel_moment = 0                                  # Wind heeling moment Nm

**STEP 3) Start the preparatory calculations**

   *1. Prediction of Self propulsion RPS*

In [3]:
############################# Preparatory Calculations ###############################
#                                                                                    #
#     This section includes some basic set of calculations which must be             #
#     carried out prior to the main simulation.                                      #
#                                                                                    #
######################################################################################

print("\n Simulation started")

# 1. Prediction of Self Propulsion RPS from Froude number

Vs = Fn * sqrt(g*L)

n_min = 1                                               # Define a minimum and maximum value for Propeller RPM
n_max = 50

def self_prop_rps(n):                                   # Define the function to be used inside the iterative solver

    J = (Vs*(1-wp))/(n*Dp)

    KT = math_op.cspline_interp(J,J_pts, KT_pts)        # Interpolate the KT value for J

    T = N_prop * KT * ρ * n**2 * Dp**4                  # Calculate the Thrust

    R = math_op.cspline_interp(Vs,Vs_pts, R_pts)        # Interpolate the Resistance

    Net_thrust = (T*(1-tp)) - R

    return Net_thrust

n = math_op.itr_sol_search(self_prop_rps, n_min, n_max)   # Use iterative solver to find the solution

print("\n Still water propeller RPM =", round(n,3))


 Simulation started

 Still water propeller RPM = 3.713


*2. Calculation of still water equilibrium under action of constant wind*

In [4]:
# 2. Calculation of the still water equilibrium with action of constant wind.

GZ_max = np.amax(GZ_pts)                                # Find the max and min values from the GZ data and corresponding φ
GZ_min = np.amin(GZ_pts)                                #
φ_max  = φ_pts[np.argmax(GZ_pts)]                       #
φ_min  = φ_pts[np.argmin(GZ_pts)]                       #

Net_moment = (m * g * GZ_max) - wind_heel_moment

if(Net_moment<=0):

    print("\n Model has capsised under external wind ")

def still(φ_equi):                                                     # Define the function to be used inside the iterative solver

    GZ_equi = math_op.cspline_interp(φ_equi, φ_pts, GZ_pts)            # Interpolate the GZ value for the φ
   
    Net_moment = (m * g * GZ_equi) - wind_heel_moment

    return Net_moment

φ_equi = math_op.itr_sol_search(still, 0, φ_max)                       # Use iterative solver to find the solution

GZ_slope = math_op.cspline_deriv(φ_equi, φ_pts, GZ_pts) * 180/π        # Evaluate the slope of the GZ Curve at the new equilibrium angle

N_Roll_MI = (m * g * GM_T * Tφ**2)/(4 * π**2)

Om_R = sqrt( (m * g * GZ_slope) / N_Roll_MI )  

print("\n Equilibrium heel angle under external wind =", round(φ_equi,3) ," ", "Equilibrium GM =", np.around(GZ_slope, decimals=3) ," ", "\n \n New Roll Frequency", round(Om_R,3))

print("\n Simulation Ended")



 Equilibrium heel angle under external wind = 0.0   Equilibrium GM = 1.386   
 
 New Roll Frequency 0.55

 Simulation Ended
